In [55]:
import pandas as pd
import numpy as np
import re
import time
import random
import gensim
import jieba
from numpy import *
import json
from numpy import linalg as la  

In [56]:
wu=pd.read_csv('wvusermatrix.csv',encoding='gbk')#word2vec用户矩阵
wt=pd.read_csv('wvtvmatrix.csv',encoding='gbk')#word2vec节目矩阵
tu=pd.read_csv('tagusermatrix.csv',encoding='gbk')#标签用户矩阵
cu=pd.read_csv('channelusermatrix.csv',encoding='gbk')#标签用户频道矩阵
tt=pd.read_csv('tagtvmatrix.csv',encoding='gbk')#标签节目矩阵
dfr=pd.read_csv('mingzilianhe.csv',encoding='gbk')#用户看过的节目
info=pd.read_csv('info2.csv',encoding='gbk').fillna('')#节目信息


for i in range(len(tu)):
    if np.sum(tu.loc[i,:])!=0:
        tu.loc[i,:]=tu.loc[i,:]/np.sum(tu.loc[i,:])
for i in range(len(cu)):
    if np.sum(cu.loc[i,:])!=0:
        cu.loc[i,:]=cu.loc[i,:]/np.sum(cu.loc[i,:])
tu=tu+cu*0.2
tu.loc[:,'新闻']=0.0
        
typelist=['犯罪', '喜剧', '舞台艺术', '新闻', '儿童', '真人秀', '歌舞', '戏曲', '音乐', '冒险', '灾难', '恐怖', '家庭', '短片', '历史', '运动', '动画', '纪录片', '惊悚', '科幻', '战争', '动作', '健康', '脱口秀', '悬疑', '剧情', '古装', '鬼怪', '奇幻', '武侠', '同性', '体育', '西部', '传记', '爱情']
typelist=sorted(typelist)

In [57]:
rate={}
for i in range(len(info)):
    if info.loc[i,'rate']=='':
        rate[info.loc[i,'name']]=6.69
        continue
    di=json.loads(info.loc[i,'rate'])
    if di["average"]>0:
        rate[info.loc[i,'name']]=di["average"]
        print(di["average"])
    else:
        rate[info.loc[i,'name']]=6.69

5.1
9.0
5.2
7.6
6.1
8.3
4.0
9.2
7.6
6.1
4.7
8.6
7.3
5.0
5.7
5.7
8.8
8.3
5.5
9.0
6.3
8.8
8.7
7.3
9.6
5.6
5.6
5.6
5.1
9.2
9.0
8.2
8.3
6.5
7.8
6.5
4.6
2.5
6.6
7.0
7.3
6.2
6.9
6.2
8.2
7.1
7.6
8.0
6.3
8.3
7.8
7.7
9.6
3.6
7.1
7.1
6.6
6.1
6.5
6.2
3.0
6.8
5.4
8.1
4.8
4.7
7.9
9.3
8.3
6.9
7.2
6.5
4.9
8.5
5.4
4.5
6.2
9.2
7.3
3.0
5.4
5.3
6.7
4.7
4.7
8.0
6.4
8.0
8.3
9.7
8.5
8.0
8.8
6.0
7.0
3.3
6.0
7.6
6.2
6.3
4.9
6.4
6.0
4.7
3.2
3.0
6.2
7.9
6.7
6.3
5.4
3.4
8.8
7.8
3.9
8.3
9.4
5.3
7.6
7.8
8.2
6.8
9.5
3.2
6.9
7.4
3.2
7.2
8.7
4.8
7.4
4.7
5.8
7.1
5.9
6.8
4.1
4.1
6.8
5.2
8.2
7.4
7.0
8.3
6.5
5.1
8.2
4.7
6.9
6.5
7.2
7.2
9.4
6.2
7.9
4.6
3.4
4.6
7.2
9.5
4.0
6.9
4.6
7.7
8.4
5.4
7.3
5.8
6.1
5.1
7.4
7.0
8.7
2.6
9.1
7.1
4.4
5.2
7.0
8.0
5.5
8.9
7.3
7.9
3.4
9.1
6.7
4.0
4.9
6.3
8.7
7.9
5.4
5.1
6.2
6.4
8.0
6.3
3.6
6.2
6.9
6.8
3.6
8.4
8.2
8.8
7.3
5.5
5.8
7.2
8.1
4.8
8.0
7.3
4.8
7.2
5.5
8.5
3.0
6.0
7.5
7.7
3.8
6.5
6.7
7.5
6.0
7.8
8.2
7.6
9.1
9.0
8.8
6.1
5.8
5.5
8.1
4.3
6.2
3.1
6.8
4.4
6.1
8.5
5.5
4.6
8.2
6.3
3.7
9.3


In [58]:
#求余弦
def cosSim(inA,inB):
    num = np.sum(inA*inB,axis=1)
    denom = la.norm(inA)*la.norm(inB,axis=1)
    if denom.sum()==0.0:
        return num
    return np.nan_to_num(num/denom)

def getRec(u,wu=wu,wt=wt,tu=tu,tt=tt,dfr=dfr,info=info):
    reslist=[]
    ratelist=[]
    #print(u)
    #print(dfr.loc[u,'name'])
    ctlist=[]#记录出现过的标签
    if np.sum(tu.loc[u,:])==0:
        print('该用户无历史数据')
        return [],[]
    ua1=np.array(wu.loc[u,:])
    ua2=np.array(tu.loc[u,:])
    tv=[]
    a=cosSim(ua1,np.array(wt))*0.2
    b=cosSim(ua2,np.array(tt))*0.7
    for i in range(len(wt)):
        tv.append([i,a[i]])
    for i in range(len(tt)):
        tv[i][1]+=b[i]+0.01*rate[info.loc[i,'name']]
    tv=sorted(tv,key=lambda k:k[1],reverse=True)
    for t in tv[:12]:
        ctlist+=json.loads(info.loc[t[0],'info'])
        #print(info.loc[t[0],'name'],info.loc[t[0],'info'],t[1])
        reslist.append(info.loc[t[0],'name'])
        ratelist.append(t[1])  
    #第二轮推荐
    ctlist=list(set(ctlist))
    now=pd.DataFrame(np.ones((1,35)),columns=typelist)
    for c in ctlist:
        now.loc[0,c]=0.0
    ua2=np.array(tu.loc[u,:])*np.array(now.loc[0,:])[:35]
    tv=[]
    a=cosSim(ua1,np.array(wt))*0.2
    b=cosSim(ua2,np.array(tt))*0.7
    for i in range(len(wt)):
        tv.append([i,a[i]])
    for i in range(len(tt)):
        tv[i][1]=(tv[i][1]+b[i]+0.01*rate[info.loc[i,'name']])*0.9
    tv=sorted(tv,key=lambda k:k[1],reverse=True)
    for t in tv[:8]:
        #print(info.loc[t[0],'name'],info.loc[t[0],'info'],t[1])
        reslist.append(info.loc[t[0],'name'])
        ratelist.append(t[1])
    return reslist,ratelist

In [59]:
getRec(1)

E:\Applications\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


(['开心宝贝之开心星星球',
  '开心宝贝之开心大冒险',
  '阿U·兔智来了',
  '星际小蚂蚁3D儿歌系列我爱北京天安门',
  '星际小蚂蚁3D儿歌系列我的好妈妈',
  '赛尔号4战神风云决',
  '宝狄与好友之梦想联盟',
  '开心宝贝之古灵星历险记',
  '星际小蚂蚁3D儿歌系列世上只有妈妈好',
  '星际小蚂蚁3D儿歌系列我们多么幸福',
  '新葫芦兄弟',
  '小公主苏菲亚2',
  '你好，是鹿晗吗 2016',
  '中华好故事',
  '维多利亚的秘密时装内衣秀2016',
  '我想见到你',
  '160811-02才艺大赛决赛现场视频-高清',
  '穿越吧厨房2017',
  '我们相爱吧之爱有天意',
  '音悦V榜内地'],
 [0.65892777066035024,
  0.65219613479537697,
  0.65108748457709853,
  0.64691153753984354,
  0.64606529288080095,
  0.64573785290664742,
  0.64515564763739053,
  0.64479369506654127,
  0.64434669476885731,
  0.64246508952592107,
  0.64193660919269091,
  0.64149908069332051,
  0.59899611301504663,
  0.59721911909455172,
  0.59543461263047315,
  0.58808335548539736,
  0.58784420525180814,
  0.58750655305795541,
  0.58660633250297523,
  0.58611592784399458])

In [60]:
jg=pd.DataFrame(columns=['用户','产品名称','推荐指数'])
precision=0
cp=0
recall=0
cr=0
n=0
for i in range(len(tu)):  
    print(i)
    tuilist,ratelist=getRec(i)
    if len(tuilist)==0:
        continue
    countp=0
    countr=0
    for j in range(20):
        jg.loc[n,'用户']=i+10001
        jg.loc[n,'产品名称']=tuilist[j]
        jg.loc[n,'推荐指数']=ratelist[j]
        n+=1
#print('总准确率 召回率',precision/cp,recall/cr)

0
1
2

E:\Applications\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys



3
4
5
该用户无历史数据
6
该用户无历史数据
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
该用户无历史数据
25
26
27
28
29
30
31
32
33
34
35
36
37
38
该用户无历史数据
39
40
41
42
43
该用户无历史数据
44
45
该用户无历史数据
46
47
48
49
50
51
52
该用户无历史数据
53
54
55
该用户无历史数据
56
57
58
59
60
61
62
63
64
65
66
67
该用户无历史数据
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
该用户无历史数据
99
100
该用户无历史数据
101
102
103
104
105
106
107
108
109
该用户无历史数据
110
111
112
113
该用户无历史数据
114
115
116
117
118
119
120
121
122
123
124
125
126
该用户无历史数据
127
128
129
该用户无历史数据
130
该用户无历史数据
131
132
该用户无历史数据
133
134
135
136
137
138
139
140
141
该用户无历史数据
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
该用户无历史数据
163
164
该用户无历史数据
165
166
167
168
169
170
171
172
173
该用户无历史数据
174
175
176
177
178
179
180
181
182
183
184
该用户无历史数据
185
186
187
188
189
该用户无历史数据
190
191
192
193
该用户无历史数据
194
195
196
197
198
199
200
201
202
203
204
该用户无历史数据
205
206
207
208
209
210
211
212
213
该用户无历史数据
214
215
216
该用户无历史数据
217


1305
1306
1307
1308
1309
1310
1311
该用户无历史数据
1312
该用户无历史数据
1313
1314
该用户无历史数据
1315
1316
1317
1318
1319
1320
1321
1322
1323
1324
1325
1326
1327
1328


In [45]:
jg

,用户,产品名称,推荐指数
0,10001,小小羊提米提米的怪物,0.741918
1,10001,小老虎丹尼尔 下,0.740432
2,10001,小小羊提米提米的宠物问题,0.739525
3,10001,小小羊提米提米做音乐,0.734562
4,10001,哆啦A梦,0.731796
5,10001,小小羊提米提米的失误,0.730545
6,10001,小小羊提米提米的雪球,0.728567
7,10001,麦兜我和我妈妈,0.727363
8,10001,小小羊提米提米制造波浪,0.725226
9,10001,小小羊提米嘟嘟提米,0.725204


In [54]:
jg.to_csv('result/201800010问题一新出现推荐结果表.csv',index=False)

In [61]:
jg.to_csv('result/201800010问题一已出现推荐结果表.csv',index=False)